# Cubeviz with unusual data

This concept notebook shows Cubeviz loading:

  * GIF animation as a playable cube,
  * plain Numpy array, and
  * HST WFC3/IR ramp FITS file.

In [ ]:
from jdaviz import Cubeviz

## From GIF

The GIF is from https://media4.giphy.com/media/J5pnZ53pj4cmu30Rx5/giphy.gif. This needs `imageio` and Pillow to be installed as well.

In [ ]:
filename = '/home/lim/test_data/baby_shark/baby_shark.gif'

In [ ]:
cubeviz = Cubeviz()

In [ ]:
cubeviz.load_data(filename)

In [ ]:
cubeviz.show()

After you load the file, you can run the following cell to play the GIF animation.

In [ ]:
import time

slice_plugin = cubeviz.plugins['Slice']
interval = 0.1
n_slices = cubeviz.app.data_collection[0].shape[-1]

slice_plugin.slice = 0

for i in range(1, n_slices):
    time.sleep(interval)
    slice_plugin.slice = i

## From array

For `flux-viewer`, `(0, 0)` of first slice should have value of zero and should be at bottom left, and then subsequent values should increase in row-major order.

For `uncert-viewer`, the values are all random.

In [ ]:
import numpy as np

In [ ]:
a = np.arange(5 * 8 * 10).reshape((5, 8, 10))  # z, y, x
b = np.random.random((5, 8, 10))

In [ ]:
cubeviz = Cubeviz()

In [ ]:
cubeviz.load_data(a, data_label='ordered')
cubeviz.load_data(b, data_label='random', data_type='uncert', override_cube_limit=True)

In [ ]:
cubeviz.show()

## HST WFC3/IR Ramp (Spectrum1D, 3D)

In [ ]:
import numpy as np
from astropy import units as u
from astropy.io import fits
from astropy.nddata import StdDevUncertainty
from astropy.utils.data import download_file
from specutils import Spectrum1D

In [ ]:
wfc3ir_file = download_file('https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:HST/product/icgk01a8q_ima.fits', cache=True)
fits.info(wfc3ir_file)

In [ ]:
sci_unit = u.electron / u.s  # From BUNIT but its value is invalid for astropy.units
err_unit = sci_unit  # Assume the same

with fits.open(wfc3ir_file) as pf:
    primary_header = pf[0].header
    sci_cube = np.stack([hdu.data for hdu in pf if hdu.name == 'SCI'], axis=2) * sci_unit
    err_cube = np.stack([hdu.data for hdu in pf if hdu.name == 'ERR'], axis=2) * err_unit
    dq_cube = np.stack([hdu.data for hdu in pf if hdu.name == 'DQ'], axis=2) * u.dimensionless_unscaled
    
uncertainty = StdDevUncertainty()
uncertainty.array = err_cube

In [ ]:
sci_cube.shape

In [ ]:
sc = Spectrum1D(flux=sci_cube, uncertainty=uncertainty, mask=dq_cube, meta=primary_header)

In [ ]:
cubeviz = Cubeviz()

In [ ]:
cubeviz.load_data(sc, data_label='icgk01a8q_ima')

In [ ]:
cubeviz.show()

For sanity check, we display first slice of `SCI` in Matplotlib. Both displays should look the same.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
arr = fits.getdata(wfc3ir_file, ext=1)
arr.shape

In [ ]:
plt.imshow(arr, origin='lower', vmin=0, vmax=1000)